In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import os
%matplotlib inline
from tensorflow.python.client import device_lib
from tensorflow.keras.models import Sequential, load_model, clone_model
import tensorflow_hub as hub
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, SparseCategoricalCrossentropy, CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay, classification_report
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 5772397301478443872
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 2252026676
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 16142209529953838552
 physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5"
 xla_global_id: 416903419]

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/laxmimerit/All-CSV-ML-Data-Files-Download/master/stackoverflow.csv', index_col=0)

In [3]:
df.head()

,Text,Tags
2,aspnet site maps has anyone got experience cre...,"['sql', 'asp.net']"
4,adding scripting functionality to net applicat...,"['c#', '.net']"
5,should i use nested classes in this case i am ...,['c++']
6,homegrown consumption of web services i have b...,['.net']
8,automatically update version number i would li...,['c#']


In [4]:
df.Tags.iloc[0]

"['sql', 'asp.net']"

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48976 entries, 2 to 1263454
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Text    48976 non-null  object
 1   Tags    48976 non-null  object
dtypes: object(2)
memory usage: 1.1+ MB


In [8]:
df.isnull().sum()

Text    0
Tags    0
dtype: int64

In [57]:
from sklearn.model_selection import train_test_split

In [98]:
train_df, test_df = train_test_split(df, test_size=42, random_state=42)

In [99]:
train_df

,Text,Tags
161841,regex to match a css class name i am trying to...,"[php, css]"
706075,mach exception in iphone i sometimes get follo...,"[iphone, objective-c]"
238501,thread pool handling duplicate tasks i want to...,[java]
526709,linker errors when trying to install new googl...,[ios]
657595,not able to connect android wear emulator with...,[android]
...,...,...
79414,is there a way to wake a sleeping thread is th...,[c#]
856796,table cells height calculated differenly in ie...,[css]
535439,how to conditionally include code only if abov...,"[ios, objective-c]"
2830,how would you improve this algorithm c string ...,[c++]


In [109]:
train_df.Text.iloc[1]

'mach exception in iphone i sometimes get following exceptionmach exception xx count d code xllx xllxmach skipping registered port it is invalidmach skipping registered port mask does not matchsignal d info p uapvoid pi do not have any idea what these exceptions are about can somebody throw any light on this'

In [52]:
df.Tags = df.Tags.apply(lambda x: eval(x))

In [131]:
from sklearn.preprocessing import MultiLabelBinarizer

In [132]:
lb = MultiLabelBinarizer()

In [133]:
train_label = lb.fit_transform(train_df.Tags)

In [137]:
train_label[0]

array([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0])

In [138]:
test_label[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])

In [135]:
test_label = lb.transform(test_df.Tags)

In [68]:
import string
from re import sub

In [104]:
train_df.Text = train_df.Text.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))

In [105]:
train_df.Text = train_df.Text.apply(lambda x: sub('([0-9])', '', x))
train_df.Text = train_df.Text.apply(lambda x: x.lower())

test_df.Text = test_df.Text.apply(lambda x: ''.join([i for i in x if i not in string.punctuation]))
test_df.Text = test_df.Text.apply(lambda x: sub('([0-9])', '', x))
test_df.Text = test_df.Text.apply(lambda x: x.lower())

In [108]:
train_df.Text.iloc[1]

'mach exception in iphone i sometimes get following exceptionmach exception xx count d code xllx xllxmach skipping registered port it is invalidmach skipping registered port mask does not matchsignal d info p uapvoid pi do not have any idea what these exceptions are about can somebody throw any light on this'

In [110]:
train_text = train_df.Text.to_numpy()
test_text = test_df.Text.to_numpy()

In [111]:
train_text[0]

'regex to match a css class name i am trying to write a regex that matches a valid css class name structure i have this so farpattern azazazazsregex preg match allpattern html matcheshowever a class name can be in the following formats that my regex would not matchpmy classpthisclasthese are just some cases i have looked around to find the rules of how you can name a class in a style block but could not find anything anyone know where the rules for the class naming conventions areare there any more cases that i need to consider what regex would you use to match a class name i have already narrowed it down to a style block using the php dom document class'

In [112]:
num = [len(i.split()) for i in train_text]

In [115]:
np.mean(num)

135.88772632525442

In [120]:
np.sum(num)

6649530

In [119]:
np.percentile(num, q=90)

262.0

In [121]:
text_vector = layers.TextVectorization(max_tokens=100000, 
                                       output_sequence_length=262)

In [122]:
text_vector.adapt(train_text)

In [124]:
text_vector.get_vocabulary()[:10]

['', '[UNK]', 'the', 'i', 'to', 'a', 'is', 'in', 'and', 'of']

In [125]:
train_label.shape

(48934, 20)

## Create a model

In [214]:
# Set random seed
tf.random.set_seed(42)

# Create inputs
inputs = layers.Input(shape=(1), dtype=tf.string)

# Create text vectorizer 
text_vectorizer = layers.TextVectorization(max_tokens=100000, 
                                           output_sequence_length=262)

text_vectorizer.adapt(train_text)
x = text_vectorizer(inputs)

# Create embeddings
embeddings = layers.Embedding(input_dim=100000,
                              input_length=262,
                              output_dim=64)

x = embeddings(x)

# Create layer
x = layers.Conv1D(filters=64, kernel_size=3, padding='same')(x)
x = layers.GlobalMaxPool1D()(x)
x = layers.Dense(20, activation='sigmoid')(x)

# Pass the inputs and outputs to model
model_1 = tf.keras.Model(inputs, x)

In [215]:
model_1.summary()

Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 1)]               0         
                                                                 
 text_vectorization_10 (Text  (None, 262)              0         
 Vectorization)                                                  
                                                                 
 embedding_9 (Embedding)     (None, 262, 64)           6400000   
                                                                 
 conv1d_9 (Conv1D)           (None, 262, 64)           12352     
                                                                 
 global_max_pooling1d_7 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_9 (Dense)             (None, 20)                1300

In [216]:
model_1.compile(optimizer=Adam(),
                loss=BinaryCrossentropy(),
                metrics=['accuracy'])

In [217]:
model_1.fit(train_text, train_label,
            validation_data=(test_text, test_label),
            epochs=5)

Epoch 1/5
1530/1530 [==============================] - 19s 12ms/step - loss: 0.1468 - accuracy: 0.5024 - val_loss: 0.0913 - val_accuracy: 0.6905
Epoch 2/5
1530/1530 [==============================] - 18s 12ms/step - loss: 0.0789 - accuracy: 0.7132 - val_loss: 0.0773 - val_accuracy: 0.7381
Epoch 3/5
1530/1530 [==============================] - 18s 12ms/step - loss: 0.0622 - accuracy: 0.7640 - val_loss: 0.0754 - val_accuracy: 0.7143
Epoch 4/5
1530/1530 [==============================] - 18s 12ms/step - loss: 0.0482 - accuracy: 0.8046 - val_loss: 0.0767 - val_accuracy: 0.7143
Epoch 5/5
1530/1530 [==============================] - 18s 12ms/step - loss: 0.0357 - accuracy: 0.8342 - val_loss: 0.0873 - val_accuracy: 0.6667


In [218]:
pred = np.round(model_1.predict(['i love python and java']))

In [219]:
pred

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 0., 0., 0.]], dtype=float32)

In [220]:
lb.inverse_transform(pred)

[('java', 'python')]

In [224]:
pred = np.round(model_1.predict(['c# is most difficult language for build game']))

In [225]:
lb.inverse_transform(pred)

[('c++',)]

In [226]:
pred = np.round(model_1.predict(['databases exploration with sql and python using jupyter lab']))
lb.inverse_transform(pred)

[('python', 'sql')]

In [227]:
lb.classes_

array(['.net', 'android', 'asp.net', 'c', 'c#', 'c++', 'css', 'html',
       'ios', 'iphone', 'java', 'javascript', 'jquery', 'mysql',
       'objective-c', 'php', 'python', 'ruby', 'ruby-on-rails', 'sql'],
      dtype=object)